# WorkLog

#### Jan 30th
Started behavioral cloning project. The project uses a car simulator to collect data and test the model. The project also provides a small Flask app which interfaces between the simulator and the trained model when testing automously. I was also provided with sample data. I haven't looked at it yet. 

I spent the night collecting data by running the simulator and having the simulator record snapshots of the current state. The simulator records images from three "cameras" on the car. Front center, front left and front right. It also records the car state at the point; steering angle, throttle, break, speed. I am pretty sure I kept the car at full speed the entire time I was recording and I didn't realize there was a break and I'm not sure what trottle is so steering angle is really the only useful data point in the state. Luckily the model is only suppose to predict steering angle so this should be fine. 

I drove the car using the arrow keys rather than using a mouse because I couldn't get the beta simulator. I may come to regret this decision as I have read multiple other student reports that using the keys is a "bad data in bad data out" proposition. I have a few thoughts on this but I will come to that later. I recorded two laps on the first and second track (there are two tracks) in both directions (total of 4 recordings, 8 laps) and saved them sepretly into the data/smooth/ folder. I am hoping this is all the continuous data I need but it is relativly straight forward to create more. 

One issue the project brief points out is driving down the middle of the road all the time will not train your model to know what to do when the car starts to veer off the road. To fix this problem, the breif suggests recording the car recovering from bad situations and adding that to the training data. The only caution is you cant continuously record as you will then also train your car to weave on the road rather than driv straight and recover. The brief also suggests reading an NVIDIA [paper](http://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf) on how they trained a car. Speicically it calls out a technique by which they used the left camera with slightly more right turn angle and the right camera with a slightly more left turn to train the car to correct. This is a super logical approach and one I may eventually take. But before I make that decison I decided to take advantage of the simulator and collect data of the car recovering. This was suprisingly difficult expesially on the second course as I had to time pressing record with where I wanted the car to be on the course and then correct and then turn recording off. Its questionable if this data is any good. 

That is about all I accomplished the first day. I figured data collection would take awhile but this took longer than expected. I am starting to thing this is more of a data seletion problem than a complicated model creation problem.

I have a few early thoughts on what I might do for preprocessing, data selection, and model training. 

#### Preprocessing:
- Flip images and steering angles to double amount of data. 
- Because arrow key steering angles results in snapshots like this (0, 0, 0, 10, 0, 0, 0), consider smoothing the data by averaging the steering angle over some +/- k snapshots. I think this will only work for the continuous driving example not the file concatentated recovery snapshots. 

#### Selection:
- Use less straight road data (mentioned in NVIDIA paper)
- Use only every thrid snapshot (mentioned in NVIDIA paper)
- Create histogram to see data count by steering angle. 

#### Model:
Ideally I could use the right and left images as inputs into my model rather than use them to create extra data the way the NVIDIA paper does. If that is the case, then I am thinking of using a pretrained AlexNet model without the fully connected layers over each image (left, right, center) and then connecting them and training a fully connected layer over them. This would be a fun way to explore transferlearning and Keras which are two things I want to learn about with this project. Its possible that using a pre-trained model won't work for this project because the images come from the simulator but we will just have to see. 

I plan to use the adam optamizer and mean squared error for training. Because order doesn't matter, I should be able to shuffle all the data and split it into training and validation datasets. One suggestion I read from another student was to select three images where you expect left, center, and right respectivly and validate agains these three images to make sure your model isnt doing anythying crazy before testing in the simulatore. MSE in some ways is not a great testing metric unless your data representation is exactly how you want it. i.e. if you have too much straight data, the car mse will look better then it actually is if it is only getting straight data correct. 

Another fun way to approach this project would be to learn about recurent neural networks and train an RNN for this project. That may be more effort than it is work for this project. I do know that for one of the Udacity self driving car contests the winner trained a recurrent network and had alot of success. I am not sure how this plays with things like short snapshots for recovery training. 

#### Jan 31

##### Notes from NVIDA [paper](http://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf). 

#### System
- three cameras
- represented steering angle as 1/r there r is the turning radius in meters (I don't know what that means exactly)
- training data augmented with additional images that show the car in different shifts from the center of the lane and rotations from the direction of the road
- two shifts from left and right cameras respectivly
-  Additional shifts between the cameras and all rotations are simulated by viewpoint transformation ofthe image from the nearest camera (seems too difficult for my project)
- for aumented data, steering label for transformed images is adjusted to one that would steer the vehicle back to the desired location and orientation in two seconds.
- model runs on a single image to a steering command

#### Data
- 72 hours of draving data in a variety of driving conditions. 
- 72(hr) * 60(m/hr) * 60(s/m) * 30(frames/s) = 7,776,000 frames captured. 

#### Network
- minimize the mean squared error between the steering command output by the network and the command of either the human driver, or the adjusted steering command for off-center and rotated images
- input image is split into YUV planes
- 9 layers
        - Normalization
        - CNN(s=2x2, k=5x5, d=24)
        - CNN(s=2x2, k=5x5, d=36)
        - CNN(s=2x2, k=5x5, d=48)
        - CNN(s=1x1, k=3x3, d=64)
        - CNN(s=1x1, k=3x3, d=64)
        - FCC(d=100)
        - FCC(d=50)
        - FCC(d=10)
    
#### Data selection
- Only select data where the driver was staying in a lane and discard the rest. (not a problem in the simulator)
- To remove a bias towards driving straight the training data includes a higher proportion of frames that represent road curves.
- Sample that video at 10 FPS

#### Augmentation
- added artificial shifts and rotations to teach the network how to recover from a poor position or orientation.
- used zero mean, and the standard deviation is twice the standard deviation that we measured with human drivers.

#### Simulation
- Ran model thorugh simulator to view the output before testing on a real car.
- Manually calibrate the lane center associated with each frame in the video used by the simulator. Called this position the “ground truth” (this sounds really cool but maybe too advanced for this project)
- The simulator then modifies the next frame in the test video so that the image appears as if the vehicle were at the position that resulted by following steering commands from the CNN. This new image is then fed to the CNN and the process repeats. The simulator records the off-center distance (distance from the car to the lane center), the yaw, and the distance traveled by the virtual car. When the off-center distance exceeds one meter, a virtual human intervention is triggered, and the virtual vehicle position and orientation is reset to match the ground truth of the corresponding frame of the original test video. (Also too complicated. Also unneccessary because we have our own real simulator. It would be interesting though to use "human intervention" if car strays too far from track.)

#### Testing
autonomy = (1 − ((number of interventions) · 6 seconds) / (elapsed time [seconds])) · 100

#### Feb 1

The first network I will train will be only using the the front camera image and using all the data as is. 

#### Feb 12

So far most of the stuff I have proposed above has not worked out. I will go more in depth below, but I will first go over some of the infrastructure work I've completed. I have written some helpers for pushing data to s3 and pulling data down from s3. This is helpful for getting the driving data onto an aws ec2 instance and for getting large results (h5 files) off of aws and onto my local computer without including them in my git repo. I've written a class that takes in a dataframe and is able to return generators from the dataframe. I've written some functins to load different datasets I've been working with. I've written a function that is able to generate the putout of my training data from the convolutional layers of VGG16. 

From all of this work I felt I was fully setup to test a single center camera using all the data I'd collected and also using a smoothed over steering angle version of my data. I was able to train a simple 3 fully connected layer model on top of the VGG output and got loses arount 0.X. It was difficult to know if this was a well trained model or not so I set about trying to run the two trained models I had in the simulator. Here, I hit an unforseen error, I wasn't able to run a prediction using VGG16 on my computer's GPU. I tried using only my computer's CPU by settng the session in the Keras backend and setting the GPU to 0 for the session but I then ran into [this](https://github.com/fchollet/keras/issues/5268) but which seems like it was fixed recently but maybe not for my use case. 

Right now it feels like I won't be able to get VGG to run on my computer which pretty much blows up my entire plan for using transfer learning on this project. Its extremly disapointing because I was very excited to get some expereince with a very common practice like transfer learning on a real project. 

My plan moving forward is to try and emulate the NVIDIA paper more closely first using the sample data provided in the project. 